In [698]:
import pandas as pd
import numpy as np
import torch as th
import regex
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [699]:
df = pd.read_csv('./data/housing.csv', sep=' ', header=None, engine='python')
df = df.drop(0, axis=1)
df = df.dropna()

columns = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis',
          'rad', 'tax', 'ptratio',' black', 'lstat', 'medv']

df.columns = columns

In [701]:
df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [702]:
inputs = np.array(df.drop('medv', axis=1), dtype='float32')
inputs = th.from_numpy(inputs)
targets = np.array(df.medv.to_frame(), dtype='float32')
targets = th.from_numpy(targets)

In [703]:
train_ds = TensorDataset(inputs, targets)

In [704]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [705]:
model = nn.Linear(13, 1)
print(model.weight)
print(model.bias)
print(list(model.parameters()))

Parameter containing:
tensor([[ 0.1057,  0.0308, -0.1860,  0.0047, -0.2279, -0.2054,  0.0912,  0.2204,
          0.2365, -0.2545,  0.0422, -0.2743,  0.1677]], requires_grad=True)
Parameter containing:
tensor([-0.0182], requires_grad=True)
[Parameter containing:
tensor([[ 0.1057,  0.0308, -0.1860,  0.0047, -0.2279, -0.2054,  0.0912,  0.2204,
          0.2365, -0.2545,  0.0422, -0.2743,  0.1677]], requires_grad=True), Parameter containing:
tensor([-0.0182], requires_grad=True)]


In [706]:
preds = model(inputs)

In [707]:
loss_fn = F.mse_loss
loss = loss_fn(model(inputs.float()), targets)
print(loss)

tensor(46358.5000, grad_fn=<MseLossBackward>)


In [708]:
for xb,yb in train_dl:
    pred = model(xb.float())
    loss = loss_fn(pred, yb.float())
    loss

In [709]:
opt = th.optim.SGD(model.parameters(), lr=1e-6)

In [710]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            # 1. Generate predictions
            pred = model(xb.float())
        
            # 2. Calculate loss
            loss = loss_fn(pred, yb.float())
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [711]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 86.0737
Epoch [20/100], Loss: 26.3869
Epoch [30/100], Loss: 79.3962
Epoch [40/100], Loss: 106.3540
Epoch [50/100], Loss: 294.6284
Epoch [60/100], Loss: 399.4764
Epoch [70/100], Loss: 194.7042
Epoch [80/100], Loss: 15.0437
Epoch [90/100], Loss: 64.9493
Epoch [100/100], Loss: 25.2501
